In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module): # inherit the nn.Module class
    def __init__(self): # Class intialization
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input 28*28*1 Output: 28*28*32 RF 3*3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input 28*28*32 Output: 28*28*64 RF 5*5
        self.pool1 = nn.MaxPool2d(2, 2) #input 28*28*64 Output: 14*14*64 RF 10*10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)#input 14*14*64 Output: 14*14*128 RF 12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)#input 14*14*128 Output: 14*14*256 RF 14*14
        self.pool2 = nn.MaxPool2d(2, 2)#input 14*14*256 Output: 7*7*256 RF 28*28
        self.conv5 = nn.Conv2d(256, 512, 3)#input 7*7*256 Output: 5*5* 512RF 30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)#input 5*5*512 Output: 3*3* 1024RF 32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)#input 3*3*1024 Output: 1*1* 10RF 34*34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # conv1-Relu->cov2->Relu-Maxpool
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) # conv3-> Relu->conv4-> Relu->Maxpool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))#conv5->Relu, Conv6->Relu
        x = F.relu(self.conv7(x))#Conv> Reu
        x = x.view(-1, 10)#Flatten
        return F.log_softmax(x)#Sigmoid

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Dowln load the data and normalize it. Not sure about compose and and toTensor operation

def train(model, device, train_loader, optimizer, epoch):
    model.train() # Initialize the model for training. Initializing the weights
    pbar = tqdm(train_loader) # data loading. tqdm manages the progress bar
    for batch_idx, (data, target) in enumerate(pbar): # get the data in batch size
        data, target = data.to(device), target.to(device) #parallel loading of data and target 
        optimizer.zero_grad() # clears the old gradiants
        output = model(data) # forward prop output
        loss = F.nll_loss(output, target) # claculating loss
        loss.backward() # When you call loss.backward(), all it does is compute #gradient of loss w.r.t all the parameters in loss that have requires_grad = #True and store them in parameter.grad attribute for every parameter.
#optimizer.step() updates all the parameters based on parameter.grad
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)